In [2]:
import pandas as pd
import plotly.graph_objects as go

import yfinance as yf

import warnings
warnings.filterwarnings("ignore")

import datetime as dt

In [ ]:
# list of tickers to compare
tickers = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'TSLA', 'META', 'NVDA', 'V', 'WMT', 
           'NFLX', 'PFE', 'BABA', 'RACE', 'MC.PA', 'ZM',           
           '^DJI', '^GSPC', '^IXIC', '^RUT', '^N225', '^GDAXI', '^FCHI', '^HSI', '^FTSE',
        #    'CL=F', # crude oil, too high commission fees 
           'GC=F', 'SI=F',
           'BTC-USD', 'ETH-USD', 'DOGE-USD', 'BNB-USD', 'XRP-USD', 'SOL-USD', 'LTC-USD'
           ]

In [4]:
start_date = "2025-01-01"
end_date = dt.datetime.now().strftime("%Y-%m-%d")

# Download historical data from yf API
data = yf.download(tickers, start=start_date, end=end_date, group_by='ticker')

# download data to csv, filename have tickers joined by underscore and end_date
data.to_csv(f"assets_comparison_{end_date}.csv")

[*********************100%***********************]  34 of 34 completed


In [5]:
# Load the CSV with MultiIndex columns (Tickers, OHLCV)
df = pd.read_csv(f"assets_comparison_{end_date}.csv", header=[0,1], index_col=0)

# Drop any rows that are completely NaN (e.g. 'Date' row)
df = df.dropna(how='all')

# Convert all values to float
df = df.astype(float)

# set index as datetime
df.index = pd.to_datetime(df.index)

# keep only level 1 'Close' prices
df = df.xs('Close', level=1, axis=1)

# Show the result
df.head()

Ticker,BTC-USD,LTC-USD,XRP-USD,DOGE-USD,SOL-USD,ETH-USD,^IXIC,^FTSE,^GDAXI,SI=F,...,AMZN,^FCHI,V,AAPL,TSLA,WMT,MC.PA,PFE,^HSI,BNB-USD
Date,,,,,,,,,,,,,,,,,,,,,
2025-01-01,94419.757812,104.812088,2.322341,0.324306,193.873734,3353.504150,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,706.512939
2025-01-02,96886.875000,105.089821,2.403370,0.338610,207.767044,3451.392578,19280.789062,8260.099609,20024.660156,29.622,...,220.220001,7393.759766,311.540253,242.525177,379.279999,89.146675,620.518738,24.405708,19623.320312,705.678589
2025-01-03,98107.429688,112.687248,2.452896,0.379279,217.747330,3605.009766,19621.679688,8224.000000,19906.080078,29.806,...,224.190002,7282.220215,312.045654,242.037827,410.440002,89.919289,596.983093,24.387363,19760.269531,713.643555
2025-01-04,98236.226562,111.167595,2.419595,0.394746,216.622925,3657.706787,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,713.841187
2025-01-05,98314.960938,115.465828,2.399727,0.382563,213.393631,3634.103760,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,709.258301


In [6]:
# date filter
# keep only last year of data
one_year_ago = dt.datetime.now() - dt.timedelta(days=365)
df = df[df.index >= one_year_ago]

In [7]:
# df copies for last 6 months and 3 months of data
df_6m = df[df.index >= (dt.datetime.now() - dt.timedelta(days=182))]
df_3m = df[df.index >= (dt.datetime.now() - dt.timedelta(days=91))]
df_1m = df[df.index >= (dt.datetime.now() - dt.timedelta(days=30))]
df_1w = df[df.index >= (dt.datetime.now() - dt.timedelta(days=7))]

In [8]:
# calculate simple returns with pct_change()
simple_returns = df.pct_change().fillna(0)

# cumulative product of simple returns (correct for compounding)
cumprod_simple = (1 + simple_returns).cumprod() - 1

In [9]:
# calculate simple returns with pct_change() for all timeframes
simple_returns_6m = df_6m.pct_change().fillna(0)
simple_returns_3m = df_3m.pct_change().fillna(0)
simple_returns_1m = df_1m.pct_change().fillna(0)
simple_returns_1w = df_1w.pct_change().fillna(0)

# cumulative product of simple returns (correct for compounding) for all timeframes
cumprod_simple_6m = (1 + simple_returns_6m).cumprod() - 1
cumprod_simple_3m = (1 + simple_returns_3m).cumprod() - 1
cumprod_simple_1m = (1 + simple_returns_1m).cumprod() - 1
cumprod_simple_1w = (1 + simple_returns_1w).cumprod() - 1

In [10]:
# plot cumprod_simple
fig = go.Figure()
for column in cumprod_simple.columns:
    fig.add_trace(
        go.Scatter(
            x=cumprod_simple.index,
            y=cumprod_simple[column],
            mode="lines",
            name=column,
        )
    )

fig.update_layout(
    title="All Tickers - Cumulative Simple Returns (Last Year)",
    xaxis_title="Date",
    yaxis_title="Cumulative Return",
    legend=dict(orientation="h", yanchor="top", y=-0.15, xanchor="center", x=0.5),
    template="plotly_white",
    margin=dict(t=80, b=120),
)
fig.show()

In [11]:
# Bar plot of final cumulative returns for each ticker
final_returns = cumprod_simple.iloc[-1].dropna().sort_values()

# Create colors based on positive/negative returns
colors = ['green' if x >= 0 else 'red' for x in final_returns.values]

fig = go.Figure()
fig.add_trace(
    go.Bar(
        x=final_returns.values,
        y=final_returns.index,
        orientation='h',
        marker_color=colors,
        text=[f"{x:.1%}" for x in final_returns.values],
        textposition='outside',
    )
)

fig.update_layout(
    title="All Tickers - Cumulative Returns (Last Year)",
    xaxis_title="Cumulative Return",
    yaxis_title="Ticker",
    template="plotly_white",
    xaxis_tickformat=".0%",
    height=max(500, len(final_returns) * 20),
    margin=dict(l=200, r=80, t=80, b=60),
)
fig.show()

In [12]:
# plot last 6 months of cumprod_simple_6m returns 
fig = go.Figure()
for column in cumprod_simple_6m.columns:
    fig.add_trace(
        go.Scatter(
            x=cumprod_simple_6m.index,
            y=cumprod_simple_6m[column],
            mode="lines",
            name=column,
        )
    )

fig.update_layout(
    title="All Tickers - Cumulative Simple Returns (Last 6 Months)",
    xaxis_title="Date",
    yaxis_title="Cumulative Return",
    legend=dict(orientation="h", yanchor="top", y=-0.15, xanchor="center", x=0.5),
    template="plotly_white",
    margin=dict(t=80, b=120),
)
fig.show()

In [13]:
# Bar plot of final cumulative returns for each ticker (Last 6 Months)
final_returns_6m = cumprod_simple_6m.iloc[-1].dropna().sort_values()

# Create colors based on positive/negative returns
colors = ['green' if x >= 0 else 'red' for x in final_returns_6m.values]

fig = go.Figure()
fig.add_trace(
    go.Bar(
        x=final_returns_6m.values,
        y=final_returns_6m.index,
        orientation='h',
        marker_color=colors,
        text=[f"{x:.1%}" for x in final_returns_6m.values],
        textposition='outside',
    )
)

fig.update_layout(
    title="All Tickers - Cumulative Returns (Last 6 Months)",
    xaxis_title="Cumulative Return",
    yaxis_title="Ticker",
    template="plotly_white",
    xaxis_tickformat=".0%",
    height=max(500, len(final_returns_6m) * 20),
    margin=dict(l=200, r=80, t=80, b=60),
)
fig.show()

In [14]:
# plot last 3 months of cumprod_simple_3m returns 
fig = go.Figure()
for column in cumprod_simple_3m.columns:
    fig.add_trace(
        go.Scatter(
            x=cumprod_simple_3m.index,
            y=cumprod_simple_3m[column],
            mode="lines",
            name=column,
        )
    )

fig.update_layout(
    title="All Tickers - Cumulative Simple Returns (Last 3 Months)",
    xaxis_title="Date",
    yaxis_title="Cumulative Return",
    legend=dict(orientation="h", yanchor="top", y=-0.15, xanchor="center", x=0.5),
    template="plotly_white",
    margin=dict(t=80, b=120),
)
fig.show()

In [15]:
# Bar plot of final cumulative returns for each ticker (Last 3 Months)
final_returns_3m = cumprod_simple_3m.iloc[-1].dropna().sort_values()

# Create colors based on positive/negative returns
colors = ['green' if x >= 0 else 'red' for x in final_returns_3m.values]

fig = go.Figure()
fig.add_trace(
    go.Bar(
        x=final_returns_3m.values,
        y=final_returns_3m.index,
        orientation='h',
        marker_color=colors,
        text=[f"{x:.1%}" for x in final_returns_3m.values],
        textposition='outside',
    )
)

fig.update_layout(
    title="All Tickers - Cumulative Returns (Last 3 Months)",
    xaxis_title="Cumulative Return",
    yaxis_title="Ticker",
    template="plotly_white",
    xaxis_tickformat=".0%",
    height=max(500, len(final_returns_3m) * 20),
    margin=dict(l=200, r=80, t=80, b=60),
)
fig.show()

In [16]:
# plot last 1 month of cumprod_simple_1m returns 
fig = go.Figure()
for column in cumprod_simple_1m.columns:
    fig.add_trace(
        go.Scatter(
            x=cumprod_simple_1m.index,
            y=cumprod_simple_1m[column],
            mode="lines",
            name=column,
        )
    )

fig.update_layout(
    title="All Tickers - Cumulative Simple Returns (Last Month)",
    xaxis_title="Date",
    yaxis_title="Cumulative Return",
    legend=dict(orientation="h", yanchor="top", y=-0.15, xanchor="center", x=0.5),
    template="plotly_white",
    margin=dict(t=80, b=120),
)
fig.show()

In [17]:
# Bar plot of final cumulative returns for each ticker (Last Month)
final_returns_1m = cumprod_simple_1m.iloc[-1].dropna().sort_values()

# Create colors based on positive/negative returns
colors = ['green' if x >= 0 else 'red' for x in final_returns_1m.values]

fig = go.Figure()
fig.add_trace(
    go.Bar(
        x=final_returns_1m.values,
        y=final_returns_1m.index,
        orientation='h',
        marker_color=colors,
        text=[f"{x:.1%}" for x in final_returns_1m.values],
        textposition='outside',
    )
)

fig.update_layout(
    title="All Tickers - Cumulative Returns (Last Month)",
    xaxis_title="Cumulative Return",
    yaxis_title="Ticker",
    template="plotly_white",
    xaxis_tickformat=".0%",
    height=max(500, len(final_returns_1m) * 20),
    margin=dict(l=200, r=80, t=80, b=60),
)
fig.show()

In [18]:
# plot last 1 week of cumprod_simple_1w returns 
fig = go.Figure()
for column in cumprod_simple_1w.columns:
    fig.add_trace(
        go.Scatter(
            x=cumprod_simple_1w.index,
            y=cumprod_simple_1w[column],
            mode="lines",
            name=column,
        )
    )

fig.update_layout(
    title="All Tickers - Cumulative Simple Returns (Last Week)",
    xaxis_title="Date",
    yaxis_title="Cumulative Return",
    legend=dict(orientation="h", yanchor="top", y=-0.15, xanchor="center", x=0.5),
    template="plotly_white",
    margin=dict(t=80, b=120),
)
fig.show()

In [19]:
# Bar plot of final cumulative returns for each ticker (Last Week)
final_returns_1w = cumprod_simple_1w.iloc[-1].dropna().sort_values()

# Create colors based on positive/negative returns
colors = ['green' if x >= 0 else 'red' for x in final_returns_1w.values]

fig = go.Figure()
fig.add_trace(
    go.Bar(
        x=final_returns_1w.values,
        y=final_returns_1w.index,
        orientation='h',
        marker_color=colors,
        text=[f"{x:.1%}" for x in final_returns_1w.values],
        textposition='outside',
    )
)

fig.update_layout(
    title="All Tickers - Cumulative Returns (Last Week)",
    xaxis_title="Cumulative Return",
    yaxis_title="Ticker",
    template="plotly_white",
    xaxis_tickformat=".0%",
    height=max(500, len(final_returns_1w) * 20),
    margin=dict(l=200, r=80, t=80, b=60),
)
fig.show()